In [1]:
import sys
sys.path.append('../Util')

In [2]:
from evaluation import aabcc, sig_props, correlation, lr, perceptron, kmeans_1dim, \
                     score_comparison, run_tests, report, dimensions_report, repeated_dimensions, \
                    kmeans_multi_dim
from preparation import prepare_dataset, read_datasets

In [3]:
from scipy.stats import f_oneway

In [4]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mean_absolute_error, accuracy_score

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
import warnings
warnings.filterwarnings('ignore')

Due to big size of WE files they are not uploaded to Github, but can instead be downloaded [here](https://drive.google.com/drive/folders/10Ea62GRlq4t7bq-nK9tPtYFu0kbCciey?usp=sharing).

The code below expects a folder "Data" in the root folder containing all the information from the Google Drive.

In [7]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [8]:
labels = [m['label'] for m in models]

In [9]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_unique_pos_we.csv'
                    )

In [10]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,Number,Gender,Lemma,POS,Tense,Person
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.370462,-0.543173,0.313777,0.114062,0.105755,0.101263,0.047496,0.941036,-0.635094,0.351250,...,0.262812,-0.034462,0.003319,-0.339070,invariable,feminine,2D,NOUN,NaN,NaN
3D,0.354640,-0.545033,0.300138,0.125182,0.109215,0.103105,0.037388,0.945384,-0.627031,0.362873,...,0.266173,-0.040272,-0.011564,-0.345399,invariable,feminine,3D,NOUN,NaN,NaN
aa,0.350173,-0.535272,0.285707,0.138325,0.116959,0.096526,0.028733,0.941748,-0.610729,0.365452,...,0.266548,-0.035119,-0.017618,-0.364172,invariable,masculine,aa,NOUN,NaN,NaN
abandon,0.348776,-0.534598,0.285159,0.137568,0.115843,0.096030,0.028422,0.942982,-0.611038,0.366652,...,0.266958,-0.034558,-0.017089,-0.363497,singular,masculine,abandon,NOUN,NaN,NaN
abbaye,0.342727,-0.537595,0.289408,0.128580,0.121620,0.101963,0.029359,0.952322,-0.611914,0.365287,...,0.274898,-0.035381,-0.019363,-0.353987,singular,feminine,abbaye,NOUN,NaN,NaN


In [11]:
# There are 6 extra feautres in addition to embedding dimensions in the file: number, gender, lemma, pos, tense,
# person
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'Number'

# Methodology

1. Exclude datapoints with undefined feature values (e.g. Gender = `invariable`)
2. Encode a grammatical feature as binary (e.g. Gender = 0 if masculine and 1 if feminine) (feature vector)
3. Shuffle the data set
4. Separate the dataset into 80% "training" and 20% test data
5. For each dimension in the test dataset measure if the dimension values are dependent on the grammatic feature
* Using [ANOVA](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html): The data is split into 2 samples, all dimension values when the grammatical feature is 0 and when it's equal to 1.
* Using [Mutual Information](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html)
6. For each dimension highlighted during step #5 find medians for 2 subgroups: when the grammatical feature == 0 and when it == 1.
7. For each word in the test dataset, find the predicted label using MAE using medians of the dimensions from #6.
8. Compute accuracy on the training dataset.

We assume that achieved accuracies can be an efficient way of comparing the quality of grammatical information encoding in the word embeddings.

The experiments for `Gender` and `Number` will be performed for "nouns only", "adjectives only" and "nouns and adjectives" combined.

# Nouns

In [12]:
# We start the experiment with nouns only
pos = ['NOUN']

In [13]:
X_noun_train = []
y_noun_train = []

X_noun_test = []
y_noun_test = []

In [14]:
for we in we_with_features:
    xtr, xtst, ytr, ytst = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable') & (we.POS.isin(pos))],
                                          feature_col_count=feature_col_count,
                                          feature_name=feature,
                                          encode=True,
                                          split=True)
    X_noun_train.append(xtr)
    X_noun_test.append(xtst)
    
    y_noun_train.append(ytr)
    y_noun_test.append(ytst)

### Compute ANOVA

We split each dimension into 2 samples: feminine nouns and masculine nouns. ANOVA test is used to assess if the population means are the same.

If the population means are not the same, we can make an assumption that the gender information affects the distribution of values in the given dimension.

If p-value < 0.001, reject the null hypothesis.

In [15]:
pv_threshold = 0.01

In [16]:
anova_dims = []

In [17]:
for i in range(len(models)):
    model_dims = []
    for dim in X_noun_train[i].columns:
        sample1 = [x[0] for x in zip(X_noun_train[i][dim], y_noun_train[i]) if x[1] == 0]
        sample2 = [x[0] for x in zip(X_noun_train[i][dim], y_noun_train[i]) if x[1] == 1]
        if f_oneway(sample1, sample2).pvalue < pv_threshold:
            model_dims.append(dim)
    anova_dims.append(model_dims)

We can see that a very large amount of dimensions appear to be highlighted by the test:

In [18]:
for i in range(len(models)):
    print(f'{models[i]["label"]}:\nTotal dimensions {len(X_noun_train[i].columns)}\nANOVA dimensions: {len(anova_dims[i])}\n')

flau_small_c:
Total dimensions 512
ANOVA dimensions: 395

flau_base_u:
Total dimensions 768
ANOVA dimensions: 357

flau_base_c:
Total dimensions 768
ANOVA dimensions: 491

flau_large_c:
Total dimensions 1024
ANOVA dimensions: 851

cam_base:
Total dimensions 768
ANOVA dimensions: 26

xlm_large:
Total dimensions 1024
ANOVA dimensions: 245

xlm_base:
Total dimensions 768
ANOVA dimensions: 77

bert_base_u:
Total dimensions 768
ANOVA dimensions: 617

distilbert_base:
Total dimensions 768
ANOVA dimensions: 329

bert_base_c:
Total dimensions 768
ANOVA dimensions: 0



### Compute Mutual Information

If mutual information is 0, we can consider that a given dimension is independent from Gender information.

If MI > 0, we can't consider the dimension completely independent and it could encode the Gender information.

In [19]:
mi_dims = []

In [20]:
for i in range(len(models)):
    res = mutual_info_classif(X_noun_train[i], y_noun_train[i], discrete_features=[False]*len(X_noun_train[i].columns))
    non_indep_dims = [str(x[0]) for x in np.argwhere(res > 0)]
    mi_dims.append(non_indep_dims)

Overall, threshold of 0 finds much more dimensions. This could be potentially addressed with a different threshold.

In [21]:
for i in range(len(models)):
    print(f"""{models[i]['label']}:
    Total dimensions {len(X_noun_train[i].columns)}
    ANOVA dimensions: {len(anova_dims[i])}
    Mutual Information dimension: {len(mi_dims[i])}\n\n""")

flau_small_c:
    Total dimensions 512
    ANOVA dimensions: 395
    Mutual Information dimension: 370


flau_base_u:
    Total dimensions 768
    ANOVA dimensions: 357
    Mutual Information dimension: 430


flau_base_c:
    Total dimensions 768
    ANOVA dimensions: 491
    Mutual Information dimension: 523


flau_large_c:
    Total dimensions 1024
    ANOVA dimensions: 851
    Mutual Information dimension: 804


cam_base:
    Total dimensions 768
    ANOVA dimensions: 26
    Mutual Information dimension: 386


xlm_large:
    Total dimensions 1024
    ANOVA dimensions: 245
    Mutual Information dimension: 620


xlm_base:
    Total dimensions 768
    ANOVA dimensions: 77
    Mutual Information dimension: 353


bert_base_u:
    Total dimensions 768
    ANOVA dimensions: 617
    Mutual Information dimension: 695


distilbert_base:
    Total dimensions 768
    ANOVA dimensions: 329
    Mutual Information dimension: 531


bert_base_c:
    Total dimensions 768
    ANOVA dimensions: 0
    

For now, for each model we select only dimensions that are potentially dependent on the gender information and found by the both tests.

In [22]:
mi_annova_dims = [set(anova_dims[i]).intersection(mi_dims[i]) for i in range(len(models))]

Final number of dimensions that we can consider not independent from the gender information for each model:

In [23]:
for i in range(len(models)):
    print(f'{models[i]["label"]}: {len(mi_annova_dims[i])}')

flau_small_c: 308
flau_base_u: 217
flau_base_c: 375
flau_large_c: 708
cam_base: 13
xlm_large: 175
xlm_base: 39
bert_base_u: 564
distilbert_base: 253
bert_base_c: 0


### Compute medians

We can try making predictions using the following dimensions:
- All dimensions of WE (as the baseline)
- Using the dimensions found in the ANOVA test
- Using the dimensions found in the Mutual Information test
- Using dimensions found in the both test

To make such predictions we will:
- Compute medians of class 1 and class 0 using only selected dimensions
- For each word in the test set compute its label by finding MAE in comparison to the the median
- Assign the label of the class with the smallest MAE

In [24]:
medians_all = []

In [25]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_noun_train[i][y_noun_train[i] == 0].median()
    medians_df['median_1'] = X_noun_train[i][y_noun_train[i] == 1].median()
    medians_all.append(medians_df)

In [26]:
medians_anova = []

In [27]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_noun_train[i][y_noun_train[i] == 0][list(anova_dims[i])].median()
    medians_df['median_1'] = X_noun_train[i][y_noun_train[i] == 1][list(anova_dims[i])].median()
    medians_anova.append(medians_df)

In [28]:
medians_mi = []

In [29]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_noun_train[i][y_noun_train[i] == 0][list(mi_dims[i])].median()
    medians_df['median_1'] = X_noun_train[i][y_noun_train[i] == 1][list(mi_dims[i])].median()
    medians_mi.append(medians_df)

In [30]:
medians_combined = []

In [31]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_noun_train[i][y_noun_train[i] == 0][list(mi_annova_dims[i])].median()
    medians_df['median_1'] = X_noun_train[i][y_noun_train[i] == 1][list(mi_annova_dims[i])].median()
    medians_combined.append(medians_df)

In [32]:
medians_combined[1]

,median_0,median_1
252,0.484909,0.494005
382,0.440108,0.442751
402,0.498378,0.480110
391,0.457846,0.476440
396,0.529876,0.543103
...,...,...
710,0.543586,0.548633
503,0.523560,0.521876
15,0.464503,0.444439
642,0.499523,0.506169


### Predict label for test set using MAE

In [33]:
y_preds_all = []

In [34]:
for i in range(len(models)):
    mae0 = X_noun_test[i].apply(lambda x: mean_absolute_error(medians_all[i]['median_0'], x), axis=1)
    mae1 = X_noun_test[i].apply(lambda x: mean_absolute_error(medians_all[i]['median_1'], x), axis=1)
    # If MAE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_all.append(y_pred)

In [35]:
y_preds_anova = []

In [36]:
for i in range(len(models)):
    mae0 = X_noun_test[i][list(anova_dims[i])].apply(lambda x: mean_absolute_error(medians_anova[i]['median_0'], x), axis=1)
    mae1 = X_noun_test[i][list(anova_dims[i])].apply(lambda x: mean_absolute_error(medians_anova[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_anova.append(y_pred)

In [37]:
y_preds_mi = []

In [38]:
for i in range(len(models)):
    mae0 = X_noun_test[i][list(mi_dims[i])].apply(lambda x: mean_absolute_error(medians_mi[i]['median_0'], x), axis=1)
    mae1 = X_noun_test[i][list(mi_dims[i])].apply(lambda x: mean_absolute_error(medians_mi[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_mi.append(y_pred)

In [39]:
y_preds_combined = []

In [40]:
for i in range(len(models)):
    mae0 = X_noun_test[i][list(mi_annova_dims[i])].apply(lambda x: mean_absolute_error(medians_combined[i]['median_0'], x), axis=1)
    mae1 = X_noun_test[i][list(mi_annova_dims[i])].apply(lambda x: mean_absolute_error(medians_combined[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_combined.append(y_pred)

### Compute accuracy

In [43]:
accs_noun_df = pd.DataFrame(columns=['All dims', 'ANOVA dims', 'MI dims', 'Combined dims'], index=[m['label'] for m in models])

In [45]:
for i in range(len(models)):
    accs_noun_df.loc[models[i]['label'], 'All dims'] = accuracy_score(y_noun_test[i], y_preds_all[i])
    
    if any(y_preds_anova[i]):
        accs_noun_df.loc[models[i]['label'], 'ANOVA dims'] = accuracy_score(y_noun_test[i], y_preds_anova[i])
    else: 
        accs_noun_df.loc[models[i]['label'], 'ANOVA dims'] = 0
    
    if any(y_preds_mi[i]):
        accs_noun_df.loc[models[i]['label'], 'MI dims'] = accuracy_score(y_noun_test[i], y_preds_mi[i])
    else:
        accs_noun_df.loc[models[i]['label'], 'MI dims'] = 0
    
    if any(y_preds_combined[i]):
        accs_noun_df.loc[models[i]['label'], 'Combined dims'] = accuracy_score(y_noun_test[i], y_preds_combined[i])
    else:
        accs_noun_df.loc[models[i]['label'], 'Combined dims'] = 0

In [46]:
accs_noun_df

,All dims,ANOVA dims,MI dims,Combined dims
flau_small_c,0.901748,0.901748,0.905365,0.90657
flau_base_u,0.62064,0.614826,0.616764,0.610465
flau_base_c,0.635322,0.640747,0.648583,0.649789
flau_large_c,0.942737,0.941531,0.940928,0.940325
cam_base,0.661858,0.641118,0.661858,0.5789
xlm_large,0.549898,0.564155,0.547862,0.553971
xlm_base,0.543788,0.564155,0.543788,0.566191
bert_base_u,0.512,0.512,0.512,0.512
distilbert_base,0.627486,0.631103,0.629295,0.640145
bert_base_c,0.560579,0,0.564195,0


# Adjectives

Repeat all steps but for adjectives only.

In [47]:
pos = ['ADJ']

In [48]:
X_adj_train = []
y_adj_train = []

X_adj_test = []
y_adj_test = []

In [49]:
for we in we_with_features:
    xtr, xtst, ytr, ytst = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable') & (we.POS.isin(pos))],
                                          feature_col_count=feature_col_count,
                                          feature_name=feature,
                                          encode=True,
                                          split=True)
    X_adj_train.append(xtr)
    X_adj_test.append(xtst)
    
    y_adj_train.append(ytr)
    y_adj_test.append(ytst)

In [50]:
anova_dims_adj = []

In [51]:
for i in range(len(models)):
    model_dims = []
    for dim in X_adj_train[i].columns:
        sample1 = [x[0] for x in zip(X_adj_train[i][dim], y_adj_train[i]) if x[1] == 0]
        sample2 = [x[0] for x in zip(X_adj_train[i][dim], y_adj_train[i]) if x[1] == 1]
        if f_oneway(sample1, sample2).pvalue < pv_threshold:
            model_dims.append(dim)
    anova_dims_adj.append(model_dims)

In [52]:
mi_dims_adj = []

In [53]:
for i in range(len(models)):
    res = mutual_info_classif(X_adj_train[i], y_adj_train[i], discrete_features=[False]*len(X_adj_train[i].columns))
    non_indep_dims = [str(x[0]) for x in np.argwhere(res > 0)]
    mi_dims_adj.append(non_indep_dims)

In [54]:
mi_annova_dims_adj = [set(anova_dims_adj[i]).intersection(mi_dims_adj[i]) for i in range(len(models))]

In [55]:
for i in range(len(models)):
    print(f'{models[i]["label"]}: {len(mi_annova_dims_adj[i])}')

flau_small_c: 232
flau_base_u: 83
flau_base_c: 217
flau_large_c: 535
cam_base: 107
xlm_large: 15
xlm_base: 0
bert_base_u: 58
distilbert_base: 70
bert_base_c: 13


### Computing medians

In [56]:
medians_adj_all = []

In [57]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_adj_train[i][y_adj_train[i] == 0].median()
    medians_df['median_1'] = X_adj_train[i][y_adj_train[i] == 1].median()
    medians_adj_all.append(medians_df)

In [58]:
medians_adj_anova = []

In [59]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_adj_train[i][y_adj_train[i] == 0][list(anova_dims_adj[i])].median()
    medians_df['median_1'] = X_adj_train[i][y_adj_train[i] == 1][list(anova_dims_adj[i])].median()
    medians_adj_anova.append(medians_df)

In [60]:
medians_adj_mi = []

In [61]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_adj_train[i][y_adj_train[i] == 0][list(mi_dims_adj[i])].median()
    medians_df['median_1'] = X_adj_train[i][y_adj_train[i] == 1][list(mi_dims_adj[i])].median()
    medians_adj_mi.append(medians_df)

In [62]:
medians_adj_combined = []

In [63]:
for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_adj_train[i][y_adj_train[i] == 0][list(mi_annova_dims_adj[i])].median()
    medians_df['median_1'] = X_adj_train[i][y_adj_train[i] == 1][list(mi_annova_dims_adj[i])].median()
    medians_adj_combined.append(medians_df)

### Computing predictions

In [64]:
y_preds_adj_all = []

for i in range(len(models)):
    mae0 = X_adj_test[i].apply(lambda x: mean_absolute_error(medians_adj_all[i]['median_0'], x), axis=1)
    mae1 = X_adj_test[i].apply(lambda x: mean_absolute_error(medians_adj_all[i]['median_1'], x), axis=1)
    # If MAE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_adj_all.append(y_pred)

y_preds_adj_anova = []

for i in range(len(models)):
    mae0 = X_adj_test[i][list(anova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_adj_anova[i]['median_0'], x), axis=1)
    mae1 = X_adj_test[i][list(anova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_adj_anova[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_adj_anova.append(y_pred)

y_preds_adj_mi = []

for i in range(len(models)):
    mae0 = X_adj_test[i][list(mi_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_adj_mi[i]['median_0'], x), axis=1)
    mae1 = X_adj_test[i][list(mi_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_adj_mi[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_adj_mi.append(y_pred)

y_preds_adj_combined = []

for i in range(len(models)):
    mae0 = X_adj_test[i][list(mi_annova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_adj_combined[i]['median_0'], x), axis=1)
    mae1 = X_adj_test[i][list(mi_annova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_adj_combined[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_adj_combined.append(y_pred)

In [65]:
accs_adj_df = pd.DataFrame(columns=['All dims', 'ANOVA dims', 'MI dims', 'Combined dims'], index=[m['label'] for m in models])

In [66]:
for i in range(len(models)):
    accs_adj_df.loc[models[i]['label'], 'All dims'] = accuracy_score(y_adj_test[i], y_preds_adj_all[i])
    
    if any(y_preds_adj_anova[i]):
        accs_adj_df.loc[models[i]['label'], 'ANOVA dims'] = accuracy_score(y_adj_test[i], y_preds_adj_anova[i])
    else: 
        accs_adj_df.loc[models[i]['label'], 'ANOVA dims'] = 0
    
    if any(y_preds_adj_mi[i]):
        accs_adj_df.loc[models[i]['label'], 'MI dims'] = accuracy_score(y_adj_test[i], y_preds_adj_mi[i])
    else:
        accs_adj_df.loc[models[i]['label'], 'MI dims'] = 0
    
    if any(y_preds_adj_combined[i]):
        accs_adj_df.loc[models[i]['label'], 'Combined dims'] = accuracy_score(y_adj_test[i], y_preds_adj_combined[i])
    else:
        accs_adj_df.loc[models[i]['label'], 'Combined dims'] = 0

Here are the accuracies for adjectives only:

In [ ]:
accs_adj_df

# Adjectives and nouns

In [67]:
pos = ['ADJ', 'NOUN']

X_na_train = []
y_na_train = []

X_na_test = []
y_na_test = []

for we in we_with_features:
    xtr, xtst, ytr, ytst = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable') & (we.POS.isin(pos))],
                                          feature_col_count=feature_col_count,
                                          feature_name=feature,
                                          encode=True,
                                          split=True)
    X_na_train.append(xtr)
    X_na_test.append(xtst)
    
    y_na_train.append(ytr)
    y_na_test.append(ytst)

anova_dims_na = []

for i in range(len(models)):
    model_dims = []
    for dim in X_adj_train[i].columns:
        sample1 = [x[0] for x in zip(X_na_train[i][dim], y_na_train[i]) if x[1] == 0]
        sample2 = [x[0] for x in zip(X_na_train[i][dim], y_na_train[i]) if x[1] == 1]
        if f_oneway(sample1, sample2).pvalue < pv_threshold:
            model_dims.append(dim)
    anova_dims_na.append(model_dims)

mi_dims_na = []

for i in range(len(models)):
    res = mutual_info_classif(X_na_train[i], y_na_train[i], discrete_features=[False]*len(X_na_train[i].columns))
    non_indep_dims = [str(x[0]) for x in np.argwhere(res > 0)]
    mi_dims_na.append(non_indep_dims)

mi_annova_dims_na = [set(anova_dims_na[i]).intersection(mi_dims_na[i]) for i in range(len(models))]

for i in range(len(models)):
    print(f'{models[i]["label"]}: {len(mi_annova_dims_na[i])}')

### Computing medians

medians_na_all = []

for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_na_train[i][y_na_train[i] == 0].median()
    medians_df['median_1'] = X_na_train[i][y_na_train[i] == 1].median()
    medians_na_all.append(medians_df)

medians_na_anova = []

for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_na_train[i][y_na_train[i] == 0][list(anova_dims_na[i])].median()
    medians_df['median_1'] = X_na_train[i][y_na_train[i] == 1][list(anova_dims_na[i])].median()
    medians_na_anova.append(medians_df)

medians_na_mi = []

for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_na_train[i][y_na_train[i] == 0][list(mi_dims_na[i])].median()
    medians_df['median_1'] = X_na_train[i][y_na_train[i] == 1][list(mi_dims_na[i])].median()
    medians_na_mi.append(medians_df)

medians_na_combined = []

for i in range(len(models)):
    medians_df = pd.DataFrame(columns=['median_0', 'median_1'])
    medians_df['median_0'] = X_na_train[i][y_na_train[i] == 0][list(mi_annova_dims_na[i])].median()
    medians_df['median_1'] = X_na_train[i][y_na_train[i] == 1][list(mi_annova_dims_na[i])].median()
    medians_na_combined.append(medians_df)

### Computing predictions

y_preds_na_all = []

for i in range(len(models)):
    mae0 = X_na_test[i].apply(lambda x: mean_absolute_error(medians_na_all[i]['median_0'], x), axis=1)
    mae1 = X_na_test[i].apply(lambda x: mean_absolute_error(medians_na_all[i]['median_1'], x), axis=1)
    # If MAE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_na_all.append(y_pred)

y_preds_na_anova = []

for i in range(len(models)):
    mae0 = X_na_test[i][list(anova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_na_anova[i]['median_0'], x), axis=1)
    mae1 = X_na_test[i][list(anova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_na_anova[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_na_anova.append(y_pred)

y_preds_na_mi = []

for i in range(len(models)):
    mae0 = X_na_test[i][list(mi_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_na_mi[i]['median_0'], x), axis=1)
    mae1 = X_na_test[i][list(mi_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_na_mi[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_na_mi.append(y_pred)

y_preds_na_combined = []

for i in range(len(models)):
    mae0 = X_na_test[i][list(mi_annova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_na_combined[i]['median_0'], x), axis=1)
    mae1 = X_na_test[i][list(mi_annova_dims_adj[i])].apply(lambda x: mean_absolute_error(medians_na_combined[i]['median_1'], x), axis=1)
    # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
    y_pred = (mae0 > mae1).apply(int)
    y_preds_na_combined.append(y_pred)

accs_na_df = pd.DataFrame(columns=['All dims', 'ANOVA dims', 'MI dims', 'Combined dims'], index=[m['label'] for m in models])

for i in range(len(models)):
    accs_na_df.loc[models[i]['label'], 'All dims'] = accuracy_score(y_na_test[i], y_preds_na_all[i])
    
    if any(y_preds_adj_anova[i]):
        accs_na_df.loc[models[i]['label'], 'ANOVA dims'] = accuracy_score(y_na_test[i], y_preds_na_anova[i])
    else: 
        accs_na_df.loc[models[i]['label'], 'ANOVA dims'] = 0
    
    if any(y_preds_adj_mi[i]):
        accs_na_df.loc[models[i]['label'], 'MI dims'] = accuracy_score(y_na_test[i], y_preds_na_mi[i])
    else:
        accs_na_df.loc[models[i]['label'], 'MI dims'] = 0
    
    if any(y_preds_adj_combined[i]):
        accs_na_df.loc[models[i]['label'], 'Combined dims'] = accuracy_score(y_na_test[i], y_preds_na_combined[i])
    else:
        accs_adj_df.loc[models[i]['label'], 'Combined dims'] = 0

flau_small_c: 332
flau_base_u: 260
flau_base_c: 403
flau_large_c: 725
cam_base: 7
xlm_large: 188
xlm_base: 36
bert_base_u: 579
distilbert_base: 286
bert_base_c: 0


ValueError: Found input variables with inconsistent numbers of samples: [393, 299]